<a href="https://colab.research.google.com/github/Sofil1886/Finance/blob/main/Data_Frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-edgar
import edgar
edgar.download_index(r"from2003",2009,"ivan.sofilkanich.99@gmail.com",skip_all_present_except_last=False)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
selectedcompany = "ELECTRONIC ARTS INC"
selectedreport='10-Q'
import requests
import json
import os
files = []
import os,glob
folder_path = r'/content/from2003'
for filename in glob.glob(os.path.join(folder_path, '*tsv')):
  with open(filename, 'r') as f:
    csv = pd.read_csv(f, sep = '\t', lineterminator='\n', names=None)
    files.append(csv)

In [ ]:
Company_reports = []
for csv in files:
  csv.rename(columns={(csv.columns[0]): ('Item')}, inplace=True)
  companyreport = csv[(csv['Item'].str.contains(selectedcompany)) & (csv['Item'].str.contains(selectedreport))]
  Company_reports.append(companyreport)

In [ ]:
filings = []
for companyreport in Company_reports:
  Filing = companyreport['Item'].str.split('|')
  Filing = Filing.to_list()
  filings.append(Filing)

In [ ]:
filings[0]

[['712515',
  'ELECTRONIC ARTS INC.',
  '10-Q',
  '2022-02-08',
  'edgar/data/712515/0000712515-22-000003.txt',
  'edgar/data/712515/0000712515-22-000003-index.html']]

In [ ]:
URLS = []
for i in filings:
  for z in i:
    for x in z:
      if 'html'in x:
        report = x
        URLS.append(report)

In [ ]:
URLS_Full = []
for each in URLS:
  url = 'https://www.sec.gov/Archives/'+ each
  URLS_Full.append(url)
URLS_Full

['https://www.sec.gov/Archives/edgar/data/712515/0000712515-22-000003-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0001193125-11-025954-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0001193125-11-216530-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0001193125-10-025856-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0000712515-16-000086-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0000712515-15-000070-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0000712515-16-000149-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0000712515-17-000010-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0000712515-12-000010-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0000712515-15-000012-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0001193125-11-302274-index.html',
 'https://www.sec.gov/Archives/edgar/data/712515/0001193125-09-230789-index.html',
 'ht

In [ ]:
formatted_urls = []
document_urls = []
for url in URLS_Full:
  r = requests.get(url, headers={'user-agent': 'my-agent-0.0.1'},
                     cookies={'one': 'true'})
  df = pd.read_html(r.text)

  document_index = df[0]
  document_index = document_index.dropna()
  document_name = document_index[document_index['Description'].str.contains(selectedreport)]
  document_name = document_name['Document'].str.split(' ')
  document_name = document_name[0][0]
  report_formatted = url.replace('-','').replace('index.html', '')
#report_formatted
#url = 'https://www.sec.gov/Archives/' + report_formatted + '/' + document_name
#url
  document_urls.append(document_name)
  formatted_urls.append(report_formatted)

In [ ]:
final_urls = []
for x, y in zip(formatted_urls, document_urls):
  url = x + '/' + y
  final_urls.append(url)

In [ ]:
Balances = []
for doc in final_urls:
  r = requests.get(doc, headers={'user-agent': 'my-agent-0.0.1'},
                     cookies={'one': 'true'})
  df = pd.read_html(r.text)
  for item in df[0:15]:
      BS = (item[0].apply(str).str.contains('TOTAL LIABILITIES') | item[0].apply(str).str.contains('Total Assets'))
      if BS.any():
        Balance_Sheet = item
        Balance_Sheet.T.drop_duplicates().T
        Balance_Sheet = Balance_Sheet.iloc[1:]
  Balances.append(Balance_Sheet) 

In [ ]:
partly_formatted_balances = []
for Balance in Balances:
  BS_undub = Balance.T.drop_duplicates().T
  BS_undub.dropna(axis='columns',how='all', inplace=True)
  for column in BS_undub.columns:
    USD_delete = BS_undub[column].apply(str) == '$'  
    bitch = BS_undub[column].apply(str) == ')'
    change = BS_undub[column].apply(str) == '$ Change'
    if USD_delete.any():
      BS_undub.pop(column)
    elif bitch.any():
      BS_undub.pop(column)
    elif change.any():
      BS_undub.pop(column)
  partly_formatted_balances.append(BS_undub)



new = []
for partly_formatted in partly_formatted_balances:
  partly_formatted[partly_formatted.columns[1:]] = partly_formatted[partly_formatted.columns[1:]].astype(str)
  NaN = partly_formatted.iloc[0] == 'nan'
  if NaN.any():
     partly_formatted = partly_formatted.iloc[1:]
  new.append(partly_formatted)

new[1]


,0,3,7
1,"(Unaudited) (In millions, except par value data)","December 31,2010","March 31,2010 (a)"
2,ASSETS,nan,nan
3,Current assets:,nan,nan
4,Cash and cash equivalents,1353,1273
5,Short-term investments,511,432
6,Marketable equity securities,107,291
7,"Receivables, net of allowances of $336 and $21...",390,206
8,Inventories,105,100
9,"Deferred income taxes, net",22,44
10,Other current assets,226,239


In [ ]:
new_names = []
for balance in new:
  header = balance.iloc[0]
  balance.columns = header
  balance.rename(columns = {(balance.columns[0]): ('Name')}, inplace=True)
  balance = balance[balance['Name'].notna()]
  balance = balance.iloc[1:]
  new_names.append(balance)

almost = []
for bal in new_names:
  df1 = bal.T.groupby(level=0).first().T
  df1.index = df1['Name']
  df2=df1.drop(columns = 'Name', axis = 1)
  almost.append(df2)


almost[2]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


1,"June 30,2011","March 31,2011 (a)"
Name,,
ASSETS,nan,nan
Current assets:,nan,nan
Cash and cash equivalents,1173,1579
Short-term investments,503,497
Marketable equity securities,172,161
"Receivables, net of allowances of $227 and $304, respectively",30,335
Inventories,75,77
"Deferred income taxes, net",58,56
Other current assets,343,327


In [ ]:
final = []
for New_Balance in almost:
  for column in New_Balance:
    New_Balance[column] = New_Balance[column].apply(str).str.replace('(', '-')
    New_Balance[column] = New_Balance[column].apply(str).str.replace(',', '')
    New_Balance[column] = New_Balance[column].apply(str).str.replace('—', 'nan')
    New_Balance[column] = New_Balance[column].apply(str).str.replace(')', '')
    New_Balance[column] = New_Balance[column].apply(str).str.replace('', 'nan')
  final.append(New_Balance)


result = []
for formatted_Balance in final:
  formatted_Balance[formatted_Balance.columns[:]] = formatted_Balance[formatted_Balance.columns[:]].astype(float)
  formatted_Balance.reset_index()
  result.append(formatted_Balance)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  import sys


In [ ]:
result[0]

1,"December 31, 2021","March 31, 2021 (a)"
Name,,
ASSETS,NaN,NaN
Current assets:,NaN,NaN
Cash and cash equivalents,2670.0,5260.0
Short-term investments,346.0,1106.0
"Receivables, net",965.0,521.0
Other current assets,377.0,326.0
Total current assets,4358.0,7213.0
"Property and equipment, net",522.0,491.0
Goodwill,5389.0,2868.0


In [ ]:
reindex_of_df = []
for reind in result:
  r = reind.reset_index()
  reindex_of_df.append(r)

In [ ]:
#new_df = pd.concat([reindex_of_df[0], reindex_of_df[1]], axis=1)
new_df = pd.merge(reindex_of_df[0], reindex_of_df[1], how="outer" ,on = 'Name')
new_df

1,Name,"December 31, 2021","March 31, 2021 (a)","December 31,2010","March 31,2010 (a)"
0,ASSETS,NaN,NaN,NaN,NaN
1,Current assets:,NaN,NaN,NaN,NaN
2,Cash and cash equivalents,2670.0,5260.0,1353.0,1273.0
3,Short-term investments,346.0,1106.0,511.0,432.0
4,"Receivables, net",965.0,521.0,NaN,NaN
5,Other current assets,377.0,326.0,226.0,239.0
6,Total current assets,4358.0,7213.0,2714.0,2585.0
7,"Property and equipment, net",522.0,491.0,502.0,537.0
8,Goodwill,5389.0,2868.0,1107.0,1093.0
9,"Acquisition-related intangibles, net",1052.0,309.0,160.0,204.0


In [ ]:
result = pd.concat([df1, df4], ignore_index=True, sort=False)
def togeather (data): 

In [ ]:
new_df = pd.concat([result[0], result[1]], axis=1)

InvalidIndexError: ignored

In [ ]:
pd.concat([df1, df4], axis=1).reindex(df1.index)

In [ ]:
writer = pd.ExcelWriter('output.xlsx')

for x in result:
  x.to_excel(writer, 'marks')
  writer.save() 
  print('DataFrame is written successfully to Excel Sheet.')


DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.
DataFrame is written successfully to Excel Sheet.


In [ ]:
print('-'*300)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [ ]:
csv = pd.read_csv(r'/content/C:\Users\Ivan\Desktop/2019-QTR4.tsv', sep = '\t', lineterminator='\n', names=None)
csv.columns.values[0] = 'Item'
companyreport = csv[(csv['Item'].str.contains(selectedcompany)) & (csv['Item'].str.contains(selectedreport))]

In [ ]:
Filing = companyreport['Item'].str.split('|')
Filing = Filing.to_list()
Filing

[['712515',
  'ELECTRONIC ARTS INC.',
  '10-Q',
  '2019-11-06',
  'edgar/data/712515/0000712515-19-000129.txt',
  'edgar/data/712515/0000712515-19-000129-index.html']]

In [ ]:
for item in Filing[0]:
  if 'html'in item:
    report = item
url = 'https://www.sec.gov/Archives/'+ report

import requests

r = requests.get(url,  headers={'user-agent': 'my-agent-0.0.1'}, cookies={'one': 'true'})

df = pd.read_html(r.text)

document_index = df[0]
document_index = document_index.dropna()


document_name = document_index[document_index['Description'].str.contains(selectedreport)]
document_name = document_name['Document'].str.split(' ')
document_name = document_name[0][0]

report_formatted = report.replace('-','').replace('index.html', '')
report_formatted
url = 'https://www.sec.gov/Archives/' + report_formatted + '/' + document_name
url

'https://www.sec.gov/Archives/edgar/data/712515/000071251519000129/ea9302019-q2fy2010qdoc.htm'

In [ ]:
r = requests.get(url, headers=header)
df = pd.read_html(r.text)
for item in df:
  BS = (item[0].str.contains('Retained') | item[0].str.contains('Total Assets'))
  if BS.any():
    Balance_Sheet = item

Balance_Sheet = Balance_Sheet.iloc[2:,[0,2,6]]
Header = Balance_Sheet.iloc[0]
Balance_Sheet = Balance_Sheet[1:]

Balance_Sheet.columns = Header

Balance_Sheet.columns[0]
Balance_Sheet.rename(columns={(Balance_Sheet.columns[0]): ('Name')}, inplace=True)

In [ ]:
Balance_Sheet = Balance_Sheet[Balance_Sheet['Name'].notna()]

Balance_Sheet[Balance_Sheet.columns[1:]] = Balance_Sheet[Balance_Sheet.columns[1:]].astype(str)

Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace('(', '-'))
Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace('(', '-'))

Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace(',', ''))
Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace(',', ''))


Balance_Sheet[Balance_Sheet.columns[1]] = Balance_Sheet[Balance_Sheet.columns[1]].map(lambda x: x.replace('—', 'nan'))
Balance_Sheet[Balance_Sheet.columns[2]] = Balance_Sheet[Balance_Sheet.columns[2]].map(lambda x: x.replace('—', 'nan'))

Balance_Sheet[Balance_Sheet.columns[1:]] = Balance_Sheet[Balance_Sheet.columns[1:]].astype(float)
Balance_Sheet

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

2,Name,"September 30, 2019","March 31, 2019 (a)"
3,ASSETS,NaN,NaN
4,Current assets:,NaN,NaN
5,Cash and cash equivalents,2940.0,4708.0
6,Short-term investments,1943.0,737.0
7,"Receivables, net",856.0,623.0
8,Other current assets,301.0,313.0
9,Total current assets,6040.0,6381.0
10,"Property and equipment, net",442.0,448.0
11,Goodwill,1890.0,1892.0
12,"Acquisition-related intangibles, net",72.0,87.0
